<a href="https://colab.research.google.com/github/HansHenseler/masdav2024/blob/main/Part_1_Data_Analysis_in_Python_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python for Data Analysis introduction

Master of Advanced Studies in Digital Forensics & Cyber Investigation

Data Analytics and Visualization for Digital Forensics

(c) Hans Henseler, 2024

This exercise starts with examples of most common data analysis tasks with Python, from the features of Python itself to using modules like Pandas with a few exercises that you can try to accomplish yourself.

# Part 1

## 1 A Note About Python Versions
All examples in this cheat sheet use Python 3. We recommend using the latest stable version of Python, for example, Python 3.8. You can check which version you have installed on your machine by running the following command in the system shell:

In [ ]:
!python --version

In [ ]:
# We need some files with sample data for the examples and excersises this morning.

from google.colab import drive
drive.mount('/content/drive')


## 2 Libraries and Imports
The easiest way to install Python modules that are needed for data analysis is to use pip. Installing NumPy and Pandas takes only a few seconds. In colab these libraries come pre installed. Once you’ve installed the modules, use the import statement to make the modules available in your program:

In [ ]:
import numpy as np
import pandas as pd


## 3 Getting Help With Python Data Analysis Functions
If you get stuck,the Google Colab interface offers context sensitive help when you are typing code. In other situations the built-in Python docs are a great place to check for tips and ways to solve the problem. The Python help() function displays the help article for a method or a class.

## 4 Working with data sources
Pandas provides a number of easy-to-use data import methods, including CSV and TSV import, copying from the system clipboard, and reading and writing JSON files. This is sufficient for most Python data analysis tasks:

In [ ]:
# Connect your drive to the Testdata folder that you have requested permission for.
# Check if the path to the file "DirPrint_Filelist - clean.xlsx" is valid. If not correct it

df = pd.read_excel('/content/drive/MyDrive/Testdata/DirPrint_Filelist - clean.xlsx')
df

## 5 Working with Pandas Data Frames
Pandas data frames are a great way to explore, clean, tweak, and filter your data sets while doing data analysis in Python. This section covers a few of the things you can do with your Pandas data frames.

# Exploring data
Here are a few functions that allow you to easily know more about the data set you are working on:#

In [ ]:
# show the data types of the columns
#
df.dtypes

In [ ]:
# show the number of columns and rows
df.shape

In [ ]:
df.info()

In [ ]:
df[:2]

In [ ]:
df[2:]

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
# we can also create a new column that is based on another colum
# The following code splits the path based on \ and stores the array in a new column called Folders
df['Folders']=df['Path'].str.split('\\')
df


## 6 Statistical operations
All standard statistical operations like minimums, maximums, and custom quantiles are present in Pandas:

In [ ]:
df.describe()

In [ ]:
df.count()

In [ ]:
df["Size"].mean()

In [ ]:
df["Size"].max()

In [ ]:
df["Size"].min()

In [ ]:
df["Size"].quantile(0.25)

In [ ]:
df["Size"].quantile(0.5)

In [ ]:
df["Size"].hist(bins=[0,100,150,100000,1000000])

In [ ]:
df["Size"].value_counts()

In [ ]:
# You can examine all values of Size by resetting the number of max_rows that are displayed
# pd.options.display.max_rows = None
# Restore this
# pd.reset_option('display.max_rows')
# The following bin sizes make more sense for this data set but the first bin
# is still relatively full

df["Size"].hist(bins=[49, 8390, 16731, 25073, 33414, 41756, 50097, 58439, 66780, 75121,
 83463, 91804, 100146, 108487, 116829, 125170, 133512])

## 7 Cleaning the Data
It is quite common to have not-a-number (NaN) values in your data set. To be able to operate on a data set with statistical methods, you’ll first need to clean up the data. The fillna and dropna Pandas functions are a convenient way to replace the NaN values with something more representative for your data set, for example, a zero, or to remove the rows with NaN values from the data frame.

In [ ]:
df.isna().sum()

In [ ]:
clean_df = df
clean_df = df["Attributes"].fillna("Unknown")

In [ ]:
clean_df

In [ ]:
df['Attributes'].dropna()

In [ ]:
df.replace('---A----','')

## 8 Reading raw text data

In step 4 we read an xlsx file. Read_xlsx nicely formats our data. This is not always the case. As an example there is a small section from the dirprint file which is formatted as tab delimited test. This file has no column names but the columns are the same as in the xlsx file used above: Attributes,Created, LastMod, LastAcc, Size, Name, Path, Folder, Ext, Md5


In [ ]:
# The DirPrint_Filelist_small.txt is just a small fragement of the xlsx file we used earlier
# It contains tab delimited data and has no headers. This is a good way to start
#
dfs = pd.read_csv('/content/drive/MyDrive/Testdata/DirPrint_Filelist_small.txt',sep='\t')
dfs.dtypes

In [ ]:
# Compare this to the dtypes we got from read_excel (see #3 above):
#
# Attributes            object
# Created       datetime64[ns]
# LastMod       datetime64[ns]
# LastAcc       datetime64[ns]
# Size                   int64
# Name                  object
# Path                  object
# Folder                object
# Ext                   object
# Md5                   object
#
# What's wrong?
#
# hint check the python docs https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
#


In [ ]:
# Add column headers
#
dfs.columns = ['Attributes','Created','LastMod','LastAcc','Size','Name','Path','Folder','Ext','Md5']
dfs.dtypes

In [ ]:
# Make the Size column an int64 (we need to take care of the , thousands separator)
#
dfs['Size'] = dfs.Size.astype(str).str.replace(',', '').astype(np.int64)
dfs.dtypes

In [ ]:
# Parse the dates
dfs['Created'] = pd.to_datetime(dfs['Created'])
dfs['LastMod'] = pd.to_datetime(dfs['LastMod'])
dfs['LastAcc'] = pd.to_datetime(dfs['LastAcc'])
dfs.dtypes

In [ ]:
# With these operations we have now cleaned/formatted the tab delimited file
# in come cases we want a column the be the index of the data frame.
#
dfs = dfs.set_index("LastMod")
dfs.dtypes

In [ ]:
dfs

In [ ]:
# If you need to repeat this process for multiple files it is also possible to
# specify this in the call to read_csv as follows
#
from datetime import datetime

# mydateparser = lambda x: pd.datetime.strptime(x, '%m/%d/%Y %H:%M')
mydateparser = lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M')

dfs = pd.read_csv(
    '/content/drive/MyDrive/Testdata/DirPrint_Filelist_small.txt',
    sep="\t",usecols=[0,1,2,3,4,5,6,7,8,9],
    names=['Attributes','Created','LastMod','LastAcc','Size','Name','Path','Folder','Ext','Md5'],
    index_col='LastMod',
    header=0,
    dtype={'Size':np.int64},
    thousands=',',
    parse_dates = ['Created', 'LastMod','LastAcc'],
    date_parser=mydateparser
    )
df.dtypes

In [ ]:
df

In [ ]:
# the datetime type also supports extracting year, month and day of the month

pd.DatetimeIndex(dfs['Created']).year

In [ ]:
pd.DatetimeIndex(dfs['Created']).month

In [ ]:
pd.DatetimeIndex(dfs['Created']).day

In [ ]:
# and even week
#
pd.DatetimeIndex(dfs['Created']).week

In [ ]:
# and even week
#
pd.DatetimeIndex(dfs['Created']).week

In [ ]:
# and even week (although the method week() was depricated and we have to use isocalendar() in stead)
#
#pd.DatetimeIndex(dfs['Created']).week
pd.DatetimeIndex(dfs['Created']).isocalendar().week

## 9 Filtering and sorting
Here are some basic commands for filtering and sorting the data in your data frames.

In [ ]:
df.sort_values(by=['Name','Size'],ascending=True)

In [ ]:
df.query('Size>1000000')

In [ ]:
# use column names as Python attributes to filter with multiple clauses
df[(df.Size>100000) & (df.Size<1000000)]

In [ ]:
# or access columns with the df[] syntax
df[(df['Size']>100000) & (df['Size']<1000000)]

In [ ]:
# we can use regular expressions to filter through the data. For more
# information see: https://docs.python.org/3/howto/regex.html
#
# For example we want to select all files with name System.*.dll where
# * is not white space. In regex this is denoted by \S
#
df[df.Name.str.match(r'(System[\S]+.dll)')==True]

## 10 Generating pivot tables

In [ ]:
# generate a pivot tabel listing the number of files and total size per extension
#
df.pivot_table(index='Ext',values='Size',aggfunc=['sum','count'])


In [ ]:
# There are actually 5 entries for zfsendtotarget. The Ext column has data in lower case and upper case
#
df[df.Ext=='ZFSendToTarget']

In [ ]:
# in order to change to lower case we have to convert the object type of column Ext to string
df['Ext']=df['Ext'].astype('|S').str.decode("utf-8")

In [ ]:
df.dtypes

In [ ]:
# now we can change the Ext colum to lower case
df['Ext']=df['Ext'].str.lower()

In [ ]:
# let's create the same pivot table as before
df.pivot_table(index='Ext',values='Size',aggfunc=['sum','count'])

In [ ]:
# the number of entries in our pivot table has been reduced to 1024 and the zfsendtotarget increased from 4 to 5

## 11 Analysing and removing duplicates


In [ ]:
# With the dataframe duplicated method we can detect duplicates in a panda dataframe
#
# also see: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html

df.duplicated('Name')

In [ ]:
# the original df has 255831 rows. Here is how we can drop rows with duplicate file names
# using the drop_duplicates method
#
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
#
df.drop_duplicates('Name', keep='last').shape

In [ ]:
# This removed more than half of our rows. We can also be more specific by defining duplicates based on multiple columns:
#
df.drop_duplicates(['Name','Size',], keep='last').shape

# Exercises

## 1 Use a regular expression to select office files based on file extension

In [ ]:
# Your answer

## 2 Remove duplicate entries in df based on the md5 value.

In [ ]:
# your answer

## 3 For a given file, find if it has duplicates and list the folders where duplicates are located

In [ ]:
# your answer

## 4 Create a pivot table showing number of files per month based on data last modified

In [ ]:
[ ] # your answer

## 5 Create a pivot table listing number of files vs file extension

In [ ]:
# your answer

## 6 Make a DirPrint file from your own computer

The DirPrint xlsx file was created with Karen's DirPrinter. This tool can be downloaded from:

https://www.karenware.com/powertools/karens-directory-printer

Download and install DirPrinter and use it to examine your own computer.
Select your own user folder and don't forget to check the box for subfolders.

In [ ]:
# your answer